In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
print("Complete")

Mounted at /content/gdrive
Complete


In [ ]:
import zipfile
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import csv
from tqdm.auto import tqdm

#Load Data

In [ ]:
csv_file = '1010408_2_4.csv'
# csv_file = '1137737.csv'
dest = '/content/gdrive/MyDrive/WPI/Research/Quick Comments/'

csv_path = os.path.join(dest, csv_file)
csv_df = pd.read_csv(csv_path)

In [ ]:
csv_df.shape

(1104, 9)

In [ ]:
csv_df = csv_df.dropna(axis=0)

#Load Model

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.5 MB 21.1 MB/s 
     |████████████████████████████████| 182 kB 89.0 MB/s 
     |████████████████████████████████| 7.6 MB 88.1 MB/s 


In [ ]:
import torch
from transformers import AutoConfig, AutoModelForSequenceClassification, TrainingArguments, Trainer
from transformers import DataCollatorWithPadding

from tqdm.auto import tqdm

In [ ]:
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
list_of_models = [
    "bigscience/bloom-560m",
    "bigscience/bloom-1b1",
    "bigscience/bloom-1b7",
    "bigscience/bloom-3b",
    "bigscience/bloom-7b1",  
    "facebook/opt-125m",
    "facebook/opt-350m"
    "facebook/opt-1.3b",
    "facebook/opt-2.7b",
    "facebook/opt-6.7b",
    "facebook/opt-13b",
    "facebook/opt-66b",
    "xlnet-large-cased",
    "tbs17/MathBERT",
    "witiko/mathberta",
    "google/flan-t5-xxl",
    "t5-base"
]

In [ ]:
training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    weight_decay=0.01,
)

In [ ]:
logs = {'model': [], 'problem_log_id': [], 'prompt':[], 'pred':[], 'actual':[]}
device = torch.device('cuda', 0)



for model_name in tqdm(list_of_models):
    # Download configuration from huggingface.co and cache.
    model = AutoModelForSequenceClassification.from_pretrained(model_name)
    tokenizer = AutoTokenizer.from_pretrained(model_name)

    def preprocess_function(examples):
        return tokenizer(examples["text"], truncation=True)

    data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_imdb["train"],
        eval_dataset=tokenized_imdb["test"],
        tokenizer=tokenizer,
        data_collator=data_collator,
    )

    for entry in tqdm(csv_df.iterrows(), total = csv_df.shape[0]):
        selected_problem = entry[1]
        prompt = selected_problem['prompt']
        output = classifier(
            prompt,
            candidate_labels=['0', '1', '2', '3', '4'],
        )

        logs['model'] = [model]
        logs['problem_log_id'] = selected_problem['problem_log_id']
        logs['prompt'] += [output['sequence']]
        logs['pred'] += [[output['scores']]]
        logs['actual'] += [selected_problem['correctness']]
        # print(f'pred: {output['scores']}, actual: {score}')


  0%|          | 0/16 [00:00<?, ?it/s]

Some weights of BloomForSequenceClassification were not initialized from the model checkpoint at bigscience/bloom-560m and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Failed to determine 'entailment' label id from the label2id mapping in the model config. Setting to -1. Define a descriptive label2id mapping in the model config to ensure correct outputs.


  0%|          | 0/1094 [00:00<?, ?it/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
/usr/local/lib/python3.7/dist-packages/transformers/pipelines/base.py:1046: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  UserWarning,
